
**Files needed to run:**

*   merged.[shp,dbf,prj,shx,cpg] --> Merged catchment area with dam attributes
*   dams_training_[1-3].csv --> Intermediate output to combine


# Set up workspace

In [ ]:
import ee
from collections import Counter
import pandas as pd
import math
from google.colab import files
import csv

In [ ]:
!pip install geopandas
!pip install descartes

     |████████████████████████████████| 1.0MB 7.7MB/s 
     |████████████████████████████████| 6.5MB 17.3MB/s 
     |████████████████████████████████| 15.3MB 287kB/s 


In [ ]:
import geopandas as gpd

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Br4hLJhv9Ai92HCaK38rFUqsro1f2I4tdLV7qyRFWhI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5KwkXA1AXBYKeDp5xfAxLmsWiKqVMcOvoEhSoCjCFcwAp0H5msmpY

Successfully saved authorization token.


# Load data

In [ ]:
#import GSW yearly data
GSW_yearly = ee.ImageCollection('JRC/GSW1_1/YearlyHistory')

In [ ]:
#define a function to remove no-data pixels
def nodata_mask(image):
  qa = image.select('waterClass')
  no_data = qa.neq(0)
  return image.updateMask(no_data)

In [ ]:
#remove all no-data pixels
yearly_processed = GSW_yearly.map(nodata_mask)

In [ ]:
#open catchment shapefile data (upload all files in '~/Analysis/catchment areas/merged_results' files first)
catchments = gpd.read_file('merged.shp')
catchments

,Feature_ID,Dam_Name,River,Main_basin,Sub_basin,Continent,Admin0,Year_Fin,Year_Const,Alt_Yr_Fin,Main_P,Sec_P,Lat,Long,Main_P_Map,intp_dist,Intp_Coord,GDAT_Coord,Correct_Co,feature__1,pre_1.0,pre_1.5,pre_2.0,pre_2.5,pre_3.0,post_1.0,post_1.5,post_2.0,post_2.5,post_3.0,pre_total,post_total,catch_area,geometry
0,5,Ain Zada,Bou Sellam,Bou Sellam,Algerian east coast,Africa,Algeria,1986,None,None,Irrigation and Water Supply,None,36.297222,5.066667,Irrigation,2.083333e-03,POINT (5.0687499999997385 36.2972222200000019),POINT (5.0666666669999998 36.2972222200000019),POINT (5.0687499999997385 36.2972222200000019),5,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.036138,"POLYGON ((5.05350 36.30786, 5.07095 36.30786, ..."
1,6,Bou Roumi,Bou Roumi,Mediterranean Coast,Algerian east coast,Africa,Algeria,1985,None,None,Irrigation,None,36.345556,2.565556,Irrigation,2.361107e-03,POINT (2.5655555560000067 36.3479166666656326),POINT (2.5655555560000001 36.3455555600000011),POINT (2.5655555560000067 36.3479166666656326),6,3.4074,0.0000,0.3015,0.0000,0.3771,1.4634,0.1467,0.4824,0.2241,1.8270,4.0860,4.1436,683.189453,"POLYGON ((2.78457 36.41258, 2.85438 36.41258, ..."
2,11,Ain Dalia,Medjerdah,Mediterranean Coast,Algerian east coast,Africa,Algeria,1987,None,None,Water Supply,None,36.262222,7.863056,Water Supply,2.361113e-03,POINT (7.8630555560000062 36.2645833333321761),POINT (7.8630555559999999 36.2622222199999982),POINT (7.8630555560000062 36.2645833333321761),11,5.4117,0.0000,0.0018,0.0000,0.0000,0.7767,0.2826,1.0143,0.0729,3.4884,5.4135,5.6349,182.316312,"POLYGON ((7.74131 36.30786, 7.75876 36.30786, ..."
3,18,Gandjelas,Cunje,South West Coast,Cuanza,Africa,Angola,1988,None,None,Hydroelectricity,None,-11.816667,17.483333,Hydroelectricity,1.237000e-12,POINT (17.4833333299990343 -11.8166666700007728),POINT (17.4833333300000007 -11.8166666700000000),POINT (17.4833333299990343 -11.8166666700007728),18,0.0279,0.0009,0.0342,0.0000,0.0054,0.1314,0.0018,0.0162,0.0000,0.0036,0.0684,0.1530,2051.623281,"POLYGON ((16.95664 -11.79341, 16.99155 -11.793..."
4,22,Cunje II,Cuanza,None,None,Africa,Angola,2004,None,None,Hydroelectricity,None,-9.793750,15.477083,Hydroelectricity,5.290000e-13,POINT (15.4770830000004196 -9.7937500000003226),POINT (15.4770830000000004 -9.7937499999999993),POINT (15.4770830000004196 -9.7937500000003226),22,445.4136,14.4324,65.2635,0.2943,101.2140,243.7326,18.4419,191.5776,0.1386,193.5819,626.6178,647.4726,107788.210710,"POLYGON ((16.64249 -9.29759, 16.65994 -9.29759..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479,35121,Zernek,Engil,Shatt al Arab,Euphrates,Europe,Turkey,1988,None,None,Irrigation,None,38.355420,43.657080,Hydroelectricity,4.737000e-12,POINT (43.6570799999959860 38.3554200000025176),POINT (43.6570800000000006 38.3554200000000023),POINT (43.6570799999959860 38.3554200000025176),35121,7.3998,0.2043,0.5517,0.0027,0.0162,0.2574,0.0369,3.0438,0.0621,4.8285,8.1747,8.2287,1684.056748,"POLYGON ((43.99179 38.52443, 44.02670 38.52443..."
5480,35123,Zeter,Tarnava Mare,None,None,Europe,Romania,1993,None,None,Flood control,None,46.466250,25.400420,Hydroelectricity,5.916127e-04,POINT (25.4000016666621633 46.4666683333290678),POINT (25.4004200000000004 46.4662500000000023),POINT (25.4000016666621633 46.4666683333290678),35123,0.1008,0.0000,0.0000,0.0000,0.0000,0.0027,0.0000,0.0108,0.0000,0.0882,0.1008,0.1017,2.600512,"POLYGON ((25.38658 46.48313, 25.40404 46.48313..."
5481,35131,Zilina,Vah,None,None,Europe,Slovakia,1998,None,None,Flood control,None,49.257080,18.682920,Hydroelectricity,1.767767e-03,POINT (18.6816700000021747 49.2558299999965925),POINT (18.6829199999999993 49.2570800000000020),POINT (18.6816700000021747 49.2558299999965925),35131,0.0090,0.0009,0.0234,0.0027,0.3033,0.0153,0.0000,0.0189,0.0009,0.3042,0.3393,0.3393,2.466502,"POLYGON ((18.66707 49.25821, 18.68452 49.25821..."
5482,35132,Ziller

In [ ]:
catchments_1_2000 = catchments.iloc[0:2000,]
catchments_2000_4000 = catchments.iloc[2000:4000,]
catchments_4000_5484 = catchments.iloc[4000:5484,]


In [ ]:
years = list(range(1984,2019)) #list of all years with data in GSW
years

# Create functions to extract water pixels

In [ ]:
def get_catchment_data(dam): #obtain catchment area of dam as GEE object
  geom = dam['geometry']
  catchment_geojson = gpd.GeoSeries(geom).__geo_interface__
  catchment_geom = catchment_geojson['features'][0]
  fc = ee.FeatureCollection([ee.Feature(catchment_geom)])
  return fc

def get_year_counts(year, fc): #obtain pixel counts of all years for a dam
  this_year = str(year)
  next_year = str(year + 1)
  year_image = ee.Image(yearly_processed.filterDate(this_year,next_year).first())
  year_result = year_image.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
  year_dict = dict(Counter(year_result.getInfo()))
  year_count = sorted(list(year_dict.items()), key = lambda x: x[0])
  return year_count

def check_year_counts(year_count): #organize pixel counts to add to results dataframe
  count_indices = [x[0] for x in year_count]
  missing_indices = list(set([1,2,3]) - set(count_indices))
  for i in missing_indices:
    empty_set_index = (i, 0)
    year_count.append(empty_set_index)
  #sort
  year_count = sorted(year_count, key = lambda x: x[0])
  list_year_count = [x[1] for x in year_count]
  return list_year_count


In [ ]:
#name all columns of results dataframe
excel_columns = ['actual_yr','feature_id',
                 '1984_1','1984_2','1984_3',
                 '1985_1','1985_2','1985_3',
                 '1986_1','1986_2','1986_3',
                 '1987_1','1987_2','1987_3',
                 '1988_1','1988_2','1988_3',
                 '1989_1','1989_2','1989_3',
                 '1990_1','1990_2','1990_3',
                 '1991_1','1991_2','1991_3',
                 '1992_1','1992_2','1992_3',
                 '1993_1','1993_2','1993_3',
                 '1994_1','1994_2','1994_3',
                 '1995_1','1995_2','1995_3',
                 '1996_1','1996_2','1996_3',
                 '1997_1','1997_2','1997_3',
                 '1998_1','1998_2','1998_3',
                 '1999_1','1999_2','1999_3',
                 '2000_1','2000_2','2000_3',
                 '2001_1','2001_2','2001_3',
                 '2002_1','2002_2','2002_3',
                 '2003_1','2003_2','2003_3',
                 '2004_1','2004_2','2004_3',
                 '2005_1','2005_2','2005_3',
                 '2006_1','2006_2','2006_3',
                 '2007_1','2007_2','2007_3',
                 '2008_1','2008_2','2008_3',
                 '2009_1','2009_2','2009_3',
                 '2010_1','2010_2','2010_3',
                 '2011_1','2011_2','2011_3',
                 '2012_1','2012_2','2012_3',
                 '2013_1','2013_2','2013_3',
                 '2014_1','2014_2','2014_3',
                 '2015_1','2015_2','2015_3',
                 '2016_1','2016_2','2016_3',
                 '2017_1','2017_2','2017_3',
                 '2018_1','2018_2','2018_3']


In [ ]:
#Debug: Check that main code works
fc = get_catchment_data(catchments.iloc[1])
dam_excel_row = [catchments.iloc[1]['Year_Fin'], catchments.iloc[1]['Feature_ID']] #set up dataframe row for dam

for year in years:
    year_count = get_year_counts(year, fc)
    list_year_count = check_year_counts(year_count) #correctly order results for each year
    dam_excel_row = dam_excel_row + list_year_count #put all years together into a row

catchments.iloc[1]['Year_Fin']
catchments.iloc[1]['Feature_ID']

In [ ]:
len(dam_excel_row)

107

# Execute code on all dams with completion year (training data)

In [ ]:
#THIS IS THE MAIN EXEC CODE

#df = pd.DataFrame(columns = excel_columns) #create dataframe of results

for index, row in catchments_1_2000.iterrows(): #iterate over each dam
  fc = get_catchment_data(row) #get catchment area of dam as GEE object
  dam_excel_row = [row['Year_Fin'], row['Feature_ID']] #set up dataframe row for dam

  for year in years:
    year_count = get_year_counts(year, fc) #get pixel counts for each year
    list_year_count = check_year_counts(year_count) #correctly order results for each year
    dam_excel_row = dam_excel_row + list_year_count #put all years together into a row

  with open('dams_training_1.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow(dam_excel_row)

  #add_df = pd.DataFrame([dam_excel_row], columns = excel_columns) #prepare row to be added to df
  #df = pd.concat([df, add_df]) #append row to the results dataframe
  print('dam ' + str(index) + ' (' + str(row['Feature_ID']) + ') completed') #track progress

files.download('dams_training_1.csv')

dam 0 (5) completed
dam 1 (6) completed
dam 2 (11) completed
dam 3 (18) completed
dam 4 (22) completed
dam 5 (25) completed
dam 6 (26) completed
dam 7 (28) completed
dam 8 (44) completed
dam 9 (51) completed
dam 10 (55) completed
dam 11 (59) completed
dam 12 (63) completed
dam 13 (76) completed
dam 14 (95) completed
dam 15 (96) completed
dam 16 (105) completed
dam 17 (108) completed
dam 18 (114) completed
dam 19 (187) completed
dam 20 (191) completed
dam 21 (205) completed
dam 22 (206) completed
dam 23 (207) completed
dam 24 (208) completed
dam 25 (209) completed
dam 26 (210) completed
dam 27 (211) completed
dam 28 (212) completed
dam 29 (226) completed
dam 30 (333) completed
dam 31 (342) completed
dam 32 (347) completed
dam 33 (348) completed
dam 34 (349) completed
dam 35 (448) completed
dam 36 (455) completed
dam 37 (459) completed
dam 38 (461) completed
dam 39 (479) completed
dam 40 (483) completed
dam 41 (484) completed
dam 42 (490) completed
dam 43 (568) completed
dam 44 (573) com

In [ ]:
#export dataframe
#df.to_csv('dams_training.csv')
#df.to_excel('dams_training.xlsx')
#files.download('dams_training.csv')
#files.download('dams_training.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for index, row in catchments_2000_4000.iterrows(): #iterate over each dam
  fc = get_catchment_data(row) #get catchment area of dam as GEE object
  dam_excel_row = [row['Year_Fin'], row['Feature_ID']] #set up dataframe row for dam

  for year in years:
    year_count = get_year_counts(year, fc) #get pixel counts for each year
    list_year_count = check_year_counts(year_count) #correctly order results for each year
    dam_excel_row = dam_excel_row + list_year_count #put all years together into a row

  with open('dams_training_2.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow(dam_excel_row)

  #add_df = pd.DataFrame([dam_excel_row], columns = excel_columns) #prepare row to be added to df
  #df = pd.concat([df, add_df]) #append row to the results dataframe
  print('dam ' + str(index) + ' (' + str(row['Feature_ID']) + ') completed') #track progress

files.download('dams_training_2.csv')

dam 2000 (23836) completed
dam 2001 (23876) completed
dam 2002 (23907) completed
dam 2003 (23930) completed
dam 2004 (23934) completed
dam 2005 (23963) completed
dam 2006 (23976) completed
dam 2007 (23989) completed
dam 2008 (23990) completed
dam 2009 (24002) completed
dam 2010 (24021) completed
dam 2011 (24039) completed
dam 2012 (24089) completed
dam 2013 (24097) completed
dam 2014 (24100) completed
dam 2015 (24102) completed
dam 2016 (24378) completed
dam 2017 (24379) completed
dam 2018 (24380) completed
dam 2019 (24386) completed
dam 2020 (24392) completed
dam 2021 (24400) completed
dam 2022 (24407) completed
dam 2023 (24409) completed
dam 2024 (24410) completed
dam 2025 (24413) completed
dam 2026 (24419) completed
dam 2027 (24423) completed
dam 2028 (24428) completed
dam 2029 (24429) completed
dam 2030 (24436) completed
dam 2031 (24438) completed
dam 2032 (24492) completed
dam 2033 (24505) completed
dam 2034 (24506) completed
dam 2035 (24509) completed
dam 2036 (24511) completed
d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for index, row in catchments_4000_5484.iterrows(): #iterate over each dam
  fc = get_catchment_data(row) #get catchment area of dam as GEE object
  dam_excel_row = [row['Year_Fin'], row['Feature_ID']] #set up dataframe row for dam

  for year in years:
    year_count = get_year_counts(year, fc) #get pixel counts for each year
    list_year_count = check_year_counts(year_count) #correctly order results for each year
    dam_excel_row = dam_excel_row + list_year_count #put all years together into a row

  with open('dams_training_3.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow(dam_excel_row)

  #add_df = pd.DataFrame([dam_excel_row], columns = excel_columns) #prepare row to be added to df
  #df = pd.concat([df, add_df]) #append row to the results dataframe
  print('dam ' + str(index) + ' (' + str(row['Feature_ID']) + ') completed') #track progress

files.download('dams_training_3.csv')

dam 4000 (11305) completed
dam 4001 (11306) completed
dam 4002 (11310) completed
dam 4003 (11311) completed
dam 4004 (11321) completed
dam 4005 (11323) completed
dam 4006 (11324) completed
dam 4007 (11327) completed
dam 4008 (11329) completed
dam 4009 (11331) completed
dam 4010 (11333) completed
dam 4011 (11334) completed
dam 4012 (11346) completed
dam 4013 (11352) completed
dam 4014 (11353) completed
dam 4015 (11360) completed
dam 4016 (11364) completed
dam 4017 (11368) completed
dam 4018 (11371) completed
dam 4019 (11373) completed
dam 4020 (11374) completed
dam 4021 (11375) completed
dam 4022 (11376) completed
dam 4023 (11389) completed
dam 4024 (11398) completed
dam 4025 (11401) completed
dam 4026 (11402) completed
dam 4027 (11408) completed
dam 4028 (11412) completed
dam 4029 (11415) completed
dam 4030 (11416) completed
dam 4031 (11417) completed
dam 4032 (11419) completed
dam 4033 (11422) completed
dam 4034 (11423) completed
dam 4035 (11429) completed
dam 4036 (11433) completed
d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Save training data

In [ ]:
# Append all excel files
with open('dams_training_all.csv', 'a') as fout:
    writer = csv.writer(fout)
    writer.writerow(excel_columns)
    
    for num in range(1,4):
      f = open("dams_training_"+str(num)+".csv")
      #f.next() # skip the header
      for line in f:
        fout.write(line)
      f.close() # not really needed

    fout.close()

# Export data
files.download('dams_training_all.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#CODE ENDS HERE

# (Test code)

In [ ]:
#ALL LINES BELOW ARE TEST CODES

In [ ]:
sample_dam = catchments.iloc[0]
geom = sample_dam['geometry']
catchment_geojson = gpd.GeoSeries(geom).__geo_interface__
catchment_geom = catchment_geojson['features'][0]

fc = ee.FeatureCollection([ee.Feature(catchment_geom)])

In [ ]:
sample_dam

Feature_ID                                                    5
Dam_Name                                               Ain Zada
River                                                Bou Sellam
Main_basin                                           Bou Sellam
Sub_basin                                   Algerian east coast
Continent                                                Africa
Admin0                                                  Algeria
Year_Fin                                                   1986
Year_Const                                                 None
Alt_Yr_Fin                                                 None
Main_P                              Irrigation and Water Supply
Sec_P                                                      None
Lat                                                     36.2972
Long                                                    5.06667
Main_P_Map                                           Irrigation
intp_dist                               

In [ ]:
year_1984 = ee.Image(yearly_processed.filterDate('1984','1985').first())
year_1985 = ee.Image(yearly_processed.filterDate('1985','1986').first())
year_1986 = ee.Image(yearly_processed.filterDate('1986','1987').first())
year_1987 = ee.Image(yearly_processed.filterDate('1987','1988').first())
year_1988 = ee.Image(yearly_processed.filterDate('1988','1989').first())
year_1989 = ee.Image(yearly_processed.filterDate('1989','1990').first())
year_1990 = ee.Image(yearly_processed.filterDate('1990','1991').first())
year_1991 = ee.Image(yearly_processed.filterDate('1991','1992').first())
year_1992 = ee.Image(yearly_processed.filterDate('1992','1993').first())
year_1993 = ee.Image(yearly_processed.filterDate('1993','1994').first())
year_1994 = ee.Image(yearly_processed.filterDate('1994','1995').first())
year_1995 = ee.Image(yearly_processed.filterDate('1995','1996').first())
year_1996 = ee.Image(yearly_processed.filterDate('1996','1997').first())
year_1997 = ee.Image(yearly_processed.filterDate('1997','1998').first())
year_1998 = ee.Image(yearly_processed.filterDate('1998','1999').first())
year_1999 = ee.Image(yearly_processed.filterDate('1999','2000').first())
year_2000 = ee.Image(yearly_processed.filterDate('2000','2001').first())
year_2001 = ee.Image(yearly_processed.filterDate('2001','2002').first())
year_2002 = ee.Image(yearly_processed.filterDate('2002','2003').first())
year_2003 = ee.Image(yearly_processed.filterDate('2003','2004').first())
year_2004 = ee.Image(yearly_processed.filterDate('2004','2005').first())
year_2005 = ee.Image(yearly_processed.filterDate('2005','2006').first())
year_2006 = ee.Image(yearly_processed.filterDate('2006','2007').first())
year_2007 = ee.Image(yearly_processed.filterDate('2007','2008').first())
year_2008 = ee.Image(yearly_processed.filterDate('2008','2009').first())
year_2009 = ee.Image(yearly_processed.filterDate('2009','2010').first())
year_2010 = ee.Image(yearly_processed.filterDate('2010','2011').first())
year_2011 = ee.Image(yearly_processed.filterDate('2011','2012').first())
year_2012 = ee.Image(yearly_processed.filterDate('2012','2013').first())
year_2013 = ee.Image(yearly_processed.filterDate('2013','2014').first())
year_2014 = ee.Image(yearly_processed.filterDate('2014','2015').first())
year_2015 = ee.Image(yearly_processed.filterDate('2015','2016').first())
year_2016 = ee.Image(yearly_processed.filterDate('2016','2017').first())
year_2017 = ee.Image(yearly_processed.filterDate('2017','2018').first())
year_2018 = ee.Image(yearly_processed.filterDate('2018','2019').first())
#year_2019 = ee.Image(yearly_processed.filterDate('2019','2020').first())

In [ ]:
result_1984 = year_1984.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1984_dict = dict(Counter(result_1984.getInfo()))
result_1984_count = sorted(list(result_1984_dict.items()), key = lambda x: x[0])

result_1985 = year_1985.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1985_dict = dict(Counter(result_1985.getInfo()))
result_1985_count = sorted(list(result_1985_dict.items()), key = lambda x: x[0])

result_1986 = year_1986.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1986_dict = dict(Counter(result_1986.getInfo()))
result_1986_count = sorted(list(result_1986_dict.items()), key = lambda x: x[0])

result_1987 = year_1987.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1987_dict = dict(Counter(result_1987.getInfo()))
result_1987_count = sorted(list(result_1987_dict.items()), key = lambda x: x[0])

result_1988 = year_1988.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1988_dict = dict(Counter(result_1988.getInfo()))
result_1988_count = sorted(list(result_1988_dict.items()), key = lambda x: x[0])

result_1989 = year_1989.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1989_dict = dict(Counter(result_1989.getInfo()))
result_1989_count = sorted(list(result_1989_dict.items()), key = lambda x: x[0])

result_1990 = year_1990.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1990_dict = dict(Counter(result_1990.getInfo()))
result_1990_count = sorted(list(result_1990_dict.items()), key = lambda x: x[0])

result_1991 = year_1991.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1991_dict = dict(Counter(result_1991.getInfo()))
result_1991_count = sorted(list(result_1991_dict.items()), key = lambda x: x[0])

result_1992 = year_1992.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1992_dict = dict(Counter(result_1992.getInfo()))
result_1992_count = sorted(list(result_1992_dict.items()), key = lambda x: x[0])

result_1993 = year_1993.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1993_dict = dict(Counter(result_1993.getInfo()))
result_1993_count = sorted(list(result_1993_dict.items()), key = lambda x: x[0])

result_1994 = year_1994.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1994_dict = dict(Counter(result_1994.getInfo()))
result_1994_count = sorted(list(result_1994_dict.items()), key = lambda x: x[0])

result_1995 = year_1995.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1995_dict = dict(Counter(result_1995.getInfo()))
result_1995_count = sorted(list(result_1995_dict.items()), key = lambda x: x[0])

result_1996 = year_1996.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1996_dict = dict(Counter(result_1996.getInfo()))
result_1996_count = sorted(list(result_1996_dict.items()), key = lambda x: x[0])

result_1997 = year_1997.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1997_dict = dict(Counter(result_1997.getInfo()))
result_1997_count = sorted(list(result_1997_dict.items()), key = lambda x: x[0])

result_1998 = year_1998.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1998_dict = dict(Counter(result_1998.getInfo()))
result_1998_count = sorted(list(result_1998_dict.items()), key = lambda x: x[0])

result_1999 = year_1999.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_1999_dict = dict(Counter(result_1999.getInfo()))
result_1999_count = sorted(list(result_1999_dict.items()), key = lambda x: x[0])

result_2000 = year_2000.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2000_dict = dict(Counter(result_2000.getInfo()))
result_2000_count = sorted(list(result_2000_dict.items()), key = lambda x: x[0])

result_2001 = year_2001.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2001_dict = dict(Counter(result_2001.getInfo()))
result_2001_count = sorted(list(result_2001_dict.items()), key = lambda x: x[0])

result_2002 = year_2002.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2002_dict = dict(Counter(result_2002.getInfo()))
result_2002_count = sorted(list(result_2002_dict.items()), key = lambda x: x[0])

result_2003 = year_2003.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2003_dict = dict(Counter(result_2003.getInfo()))
result_2003_count = sorted(list(result_2003_dict.items()), key = lambda x: x[0])

result_2004 = year_2004.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2004_dict = dict(Counter(result_2004.getInfo()))
result_2004_count = sorted(list(result_2004_dict.items()), key = lambda x: x[0])

result_2005 = year_2005.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2005_dict = dict(Counter(result_2005.getInfo()))
result_2005_count = sorted(list(result_2005_dict.items()), key = lambda x: x[0])

result_2006 = year_2006.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2006_dict = dict(Counter(result_2006.getInfo()))
result_2006_count = sorted(list(result_2006_dict.items()), key = lambda x: x[0])

result_2007 = year_2007.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2007_dict = dict(Counter(result_2007.getInfo()))
result_2007_count = sorted(list(result_2007_dict.items()), key = lambda x: x[0])

result_2008 = year_2008.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2008_dict = dict(Counter(result_2008.getInfo()))
result_2008_count = sorted(list(result_2008_dict.items()), key = lambda x: x[0])

result_2009 = year_2009.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2009_dict = dict(Counter(result_2009.getInfo()))
result_2009_count = sorted(list(result_2009_dict.items()), key = lambda x: x[0])

result_2010 = year_2010.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2010_dict = dict(Counter(result_2010.getInfo()))
result_2010_count = sorted(list(result_2010_dict.items()), key = lambda x: x[0])

result_2011 = year_2011.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2011_dict = dict(Counter(result_2011.getInfo()))
result_2011_count = sorted(list(result_2011_dict.items()), key = lambda x: x[0])

result_2012 = year_2012.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2012_dict = dict(Counter(result_2012.getInfo()))
result_2012_count = sorted(list(result_2012_dict.items()), key = lambda x: x[0])

result_2013 = year_2013.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2013_dict = dict(Counter(result_2013.getInfo()))
result_2013_count = sorted(list(result_2013_dict.items()), key = lambda x: x[0])

result_2014 = year_2014.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2014_dict = dict(Counter(result_2014.getInfo()))
result_2014_count = sorted(list(result_2014_dict.items()), key = lambda x: x[0])

result_2015 = year_2015.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2015_dict = dict(Counter(result_2015.getInfo()))
result_2015_count = sorted(list(result_2015_dict.items()), key = lambda x: x[0])

result_2016 = year_2016.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2016_dict = dict(Counter(result_2016.getInfo()))
result_2016_count = sorted(list(result_2016_dict.items()), key = lambda x: x[0])

result_2017 = year_2017.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2017_dict = dict(Counter(result_2017.getInfo()))
result_2017_count = sorted(list(result_2017_dict.items()), key = lambda x: x[0])

result_2018 = year_2018.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
result_2018_dict = dict(Counter(result_2018.getInfo()))
result_2018_count = sorted(list(result_2018_dict.items()), key = lambda x: x[0])

#result_2019 = year_2019.reduceRegion(ee.Reducer.toList(), fc, 30, None, None, False, 1e9).get('waterClass')
#result_2019_dict = dict(Counter(result_2019.getInfo()))
#result_2019_count = sorted(list(result_2019_dict.items()), key = lambda x: x[0])

In [ ]:
result_2019 = year_2019.reduceRegion(ee.Reducer.toList(), fc, 30).get('waterClass')
result_2019_dict = dict(Counter(result_2019.getInfo()))
result_2019_count = sorted(list(result_2019_dict.items()), key = lambda x: x[0])

NameError: ignored

In [ ]:
print(1984, result_1984_count)
print(1985, result_1985_count)
print(1986, result_1986_count)
print(1987, result_1987_count)
print(1988, result_1988_count)
print(1989, result_1989_count)
print(1990, result_1990_count)
print(1991, result_1991_count)
print(1992, result_1992_count)
print(1993, result_1993_count)
print(1994, result_1994_count)
print(1995, result_1995_count)
print(1996, result_1996_count)
print(1997, result_1997_count)
print(1998, result_1998_count)
print(1999, result_1999_count)
print(2000, result_2000_count)
print(2001, result_2001_count)
print(2002, result_2002_count)
print(2003, result_2003_count)
print(2004, result_2004_count)
print(2005, result_2005_count)
print(2006, result_2006_count)
print(2007, result_2007_count)
print(2008, result_2008_count)
print(2009, result_2009_count)
print(2010, result_2010_count)
print(2011, result_2011_count)
print(2012, result_2012_count)
print(2013, result_2013_count)
print(2014, result_2014_count)
print(2015, result_2015_count)
print(2016, result_2016_count)
print(2017, result_2017_count)
print(2018, result_2018_count)

1984 []
1985 []
1986 []
1987 []
1988 []
1989 []
1990 []
1991 []
1992 []
1993 []
1994 []
1995 []
1996 []
1997 []
1998 []
1999 []
2000 []
2001 []
2002 []
2003 []
2004 []
2005 []
2006 []
2007 []
2008 []
2009 []
2010 []
2011 []
2012 []
2013 []
2014 []
2015 []
2016 []
2017 []
2018 []


In [ ]:
[x[0] for x in result_2018_count]

[1, 2]

In [ ]:
#sum up the 2's and 3's to get count of water pixels and no-water pixels (1)
#and then look at the years with the biggest changes or when the changes begin to occur

In [ ]:
#Dam #2468 (Cabeça de boi, Apiacás river) — a series of three run-of-river projects
#Complications: one part of a hydroelectric complex that started construction in 2014
#and completed in 2016
#Since it is run-of-the-river, there might not be that much storage caused by the
#dam, and hence water-changes might not be very significant.
#2469 (da Fazenda) is also part of the complex

In [ ]:
#loop through all of the dams — maintain current structure of code
#but automate the process to go through all rows of the shapefile
#create a dataframe with dams as rows and years*pixels for columns: three columns per year
#i.e. 1984-1, 1984-2, 1984-3 — don't add the 2's and 3's yet

In [ ]:
sample_dam = catchments.iloc[0]
geom = sample_dam['geometry']
catchment_geojson = gpd.GeoSeries(geom).__geo_interface__
catchment_geom = catchment_geojson['features'][0]

fc = ee.FeatureCollection([ee.Feature(catchment_geom)])

before_start = row['pre_init_f']
before_end = row['pre_end']
after_start = row['post_init']
after_end = row['post_end_f']

range_predam = yearly_processed.filterDate(before_start,before_end)
range_postdam = yearly_processed.filterDate(after_start,after_end)

median_predam = range_predam.reduce(ee.Reducer.median());
median_postdam = range_postdam.reduce(ee.Reducer.median());

pre_result = median_predam.reduceRegion(ee.Reducer.toList(), fc, 40).get('waterClass_median')
pre_medians = pre_result.getInfo()

pre_medians_dict = dict(Counter(pre_medians))
pre_medians_count = sorted(list(pre_medians_dict.items()), key = lambda x: x[0])

post_result = median_postdam.reduceRegion(ee.Reducer.toList(), fc, 40).get('waterClass_median')
post_medians = post_result.getInfo()

post_medians_dict = dict(Counter(post_medians))
post_medians_count = sorted(list(post_medians_dict.items()), key = lambda x: x[0])

counts = [pre_medians_count, post_medians_count]
print(counts)


EEException: ignored

In [ ]:
#sample data
pre_medians_count = [(1, 2234),(2, 23),(3,321)]
post_medians_count = [(1, 98),(1.5, 97),(2, 897),(2.5, 7),(3, 998)]

In [ ]:
columns = ['feature_ID',
           'pre_1.0','pre_1.5','pre_2.0','pre_2.5','pre_3.0',
           'post_1.0','post_1.5','post_2.0','post_2.5','post_3.0']

df = pd.DataFrame(columns = columns)
pre_indices = [x[0] for x in pre_medians_count]
post_indices = [x[0] for x in post_medians_count]

full_indices = [1, 1.5, 2, 2.5, 3]

missing_pre = list(set(full_indices).difference(pre_indices))
for i in missing_pre:
  empty_set = (i, 0)
  pre_medians_count.append(empty_set)

missing_post = list(set(full_indices).difference(post_indices))
for i in missing_post:
  empty_set = (i, 0)
  post_medians_count.append(empty_set)

pre_medians_count = sorted(pre_medians_count, key = lambda x: x[0])
post_medians_count = sorted(post_medians_count, key = lambda x: x[0])
print(pre_medians_count, post_medians_count)

#pre = [x[1] for x in pre_medians_count]
#post = [x[1] for x in post_medians_count]

#add = [1] + pre + post
#add = pd.DataFrame([add], columns = columns)

#df = pd.concat([df, add])
#df

[(1, 2234), (1.5, 0), (2, 23), (2.5, 0), (3, 321)] [(1, 98), (1.5, 97), (2, 897), (2.5, 7), (3, 998)]


In [ ]:
df

,feature_ID,pre_1.0,pre_1.5,pre_2.0,pre_2.5,pre_3.0,post_1.0,post_1.5,post_2.0,post_2.5,post_3.0
0,5,1954,84,241,68,210,891,15,376,14,1284
0,6,1954,84,241,68,210,891,15,376,14,1284
0,11,1954,84,241,68,210,891,15,376,14,1284
0,18,1954,84,241,68,210,891,15,376,14,1284


In [ ]:
#create "before dam" and "after dam" GSW data from intervals
range_predam = yearly_processed.filterDate(before_start,before_end)
range_postdam = yearly_processed.filterDate(after_start,after_end)

In [ ]:
#calculate median water coverage of "before dam" and "after dam"
median_predam = range_predam.reduce(ee.Reducer.median());
median_postdam = range_postdam.reduce(ee.Reducer.median());

In [ ]:
#catchment area bounds
bounds = ee.FeatureCollection("users/xg2180/Ataturk_catchment");

In [ ]:
#get list of median water pixels for "before" period
#pre_result = median_predam.reduceRegion(ee.Reducer.toList(), bounds, 30, bestEffort = True).get('waterClass_median')
pre_result = median_predam.reduceRegion(ee.Reducer.toList(), bounds, 40).get('waterClass_median')
pre_medians = pre_result.getInfo()

In [ ]:
#obtain count of each pixel value in "before" period
pre_medians_count = dict(Counter(pre_medians))
print(pre_medians_count)

{1: 601954, 1.5: 1811, 2.5: 626, 2: 7546, 3: 35517}


In [ ]:
#get list of median water pixels for "after" period
#post_result = median_postdam.reduceRegion(ee.Reducer.toList(), bounds, 30, bestEffort = True).get('waterClass_median')
post_result = median_postdam.reduceRegion(ee.Reducer.toList(), bounds, 40).get('waterClass_median')
post_medians = post_result.getInfo()

In [ ]:
#obtain count of each pixel value in "after" period
post_medians_count = dict(Counter(post_medians))
print(post_medians_count)

{1: 278097, 2: 114157, 3: 249645, 2.5: 3190, 1.5: 1325}
